LCD 

#### 運用 LCD setup codes

In [7]:
import RPi.GPIO as GPIO
from time import sleep
from subprocess import check_output

GPIO.setmode(GPIO.BOARD)
    
#def pin_assignment():
LCD_RS = 38
LCD_RW = 40
LCD_EN = 29
LCD_D4 = 31
LCD_D5 = 33
LCD_D6 = 35
LCD_D7 = 37
CMD_DELAY = 0.0002
CMD_CLEAR_DELAY = 0.001

def set_GPIOs_write_mode():
    GPIO.setup(LCD_EN, GPIO.OUT) # EN
    GPIO.setup(LCD_RS, GPIO.OUT) # RS
    GPIO.setup(LCD_RW, GPIO.OUT) # RW
    GPIO.setup(LCD_D4, GPIO.OUT) # DB4
    GPIO.setup(LCD_D5, GPIO.OUT) # DB5
    GPIO.setup(LCD_D6, GPIO.OUT) # DB6
    GPIO.setup(LCD_D7, GPIO.OUT) # DB7

def RW_ready():
    sleep(0.000001)
    GPIO.output(LCD_EN, 1)
    sleep(0.000001)
    GPIO.output(LCD_EN, 0)
    
def write_lcdcmd(cmd):
    GPIO.output(LCD_EN, GPIO.LOW) 
    GPIO.output(LCD_RS, GPIO.LOW)#tell LCD this is to send commmand
    GPIO.output(LCD_RW, GPIO.LOW)
    
    #WRITE higher 4 bits
    GPIO.output(LCD_D7,GPIO.HIGH if (0b10000000 & cmd) else GPIO.LOW) #if 7th bit is set, output 1, else set 0 
    GPIO.output(LCD_D6,GPIO.HIGH if (0b01000000 & cmd) else GPIO.LOW)
    GPIO.output(LCD_D5,GPIO.HIGH if (0b00100000 & cmd) else GPIO.LOW)
    GPIO.output(LCD_D4,GPIO.HIGH if (0b00010000 & cmd) else GPIO.LOW)
    RW_ready()
    
    #WRITE lower 4 bits
    GPIO.output(LCD_D7,GPIO.HIGH if (0b00001000 & cmd) else GPIO.LOW) #if 3th bit is set, output 1, else set 0 
    GPIO.output(LCD_D6,GPIO.HIGH if (0b00000100 & cmd) else GPIO.LOW)
    GPIO.output(LCD_D5,GPIO.HIGH if (0b00000010 & cmd) else GPIO.LOW)
    GPIO.output(LCD_D4,GPIO.HIGH if (0b00000001 & cmd) else GPIO.LOW)
    RW_ready()
    sleep(CMD_DELAY)
    
def write_data(cmd):
    GPIO.output(LCD_RS, GPIO.HIGH)#tell LCD this is to send data 
    GPIO.output(LCD_EN, GPIO.LOW) 
    GPIO.output(LCD_RW, GPIO.LOW)
    
    #WRITE higher 4 bits
    GPIO.output(LCD_D7,GPIO.HIGH if (0b10000000 & cmd) else GPIO.LOW) #if 7th bit is set, output 1, else set 0 
    GPIO.output(LCD_D6,GPIO.HIGH if (0b01000000 & cmd) else GPIO.LOW)
    GPIO.output(LCD_D5,GPIO.HIGH if (0b00100000 & cmd) else GPIO.LOW)
    GPIO.output(LCD_D4,GPIO.HIGH if (0b00010000 & cmd) else GPIO.LOW)
    RW_ready()
    
    #WRITE lower 4 bits
    GPIO.output(LCD_D7,GPIO.HIGH if (0b00001000 & cmd) else GPIO.LOW) #if 3th bit is set, output 1, else set 0 
    GPIO.output(LCD_D6,GPIO.HIGH if (0b00000100 & cmd) else GPIO.LOW)
    GPIO.output(LCD_D5,GPIO.HIGH if (0b00000010 & cmd) else GPIO.LOW)
    GPIO.output(LCD_D4,GPIO.HIGH if (0b00000001 & cmd) else GPIO.LOW)
    RW_ready()
    sleep(CMD_DELAY)
    

def LCD_boot_setup():
    #1. EN,RS,RW,D7~D4 都設為 low。
    GPIO.output(LCD_D4,0)
    GPIO.output(LCD_D5,0)
    GPIO.output(LCD_D6,0)
    GPIO.output(LCD_D7,0)
    GPIO.output(LCD_RS,0)
    GPIO.output(LCD_RW,0)
    GPIO.output(LCD_EN,0)
    sleep(0.1)
    
    #2. EN 設 high -> 等一下 -> EN 設 low
    GPIO.output(LCD_EN,1)
    sleep(0.000001) #wait at least 150ns
    GPIO.output(LCD_EN,0)
    sleep(0.000001) #wait at least 150ns
    
    
    #3.D7~D4分別設為 low, low, high, low (0b0011)
    sleep(0.002)
    write_lcdcmd(0x03)
    sleep(0.001) #wait at least 4.1 ms
    write_lcdcmd(0x03)
    sleep(0.0001) #wait at least 100 us
    write_lcdcmd(0x03)
    sleep(0.0001)
    write_lcdcmd(0x02)
    sleep(0.0001)
    # function setup 
    write_lcdcmd(0x28) #0010 - set 4bit mode and 1000 - set 2 display lines and 5x8 dot font 設定為4bit操作，兩行顯示模式
    write_lcdcmd(0x0c) # 設定為每寫入一次資料，游標位置向右移一格 cursor/display shift
    write_lcdcmd(0x01) #清除螢幕並游標歸回原點  CLEAR DISPLAY
    sleep(CMD_CLEAR_DELAY) 
    write_lcdcmd(0x06) #Enable mode set
    

    
    
set_GPIOs_write_mode()
LCD_boot_setup()
    
    



### 練習 在LCM模組上第一行和第二行顯示Hello Python 

In [6]:
write_lcdcmd(0x80)
write_data(ord('H'))
write_data(ord('e'))
write_data(ord('l'))
write_data(ord('l'))
write_data(ord('o'))
write_data(ord(' '))
write_data(ord('P'))
write_data(ord('y'))
write_data(ord('t'))
write_data(ord('h'))
write_data(ord('o'))
write_data(ord('n'))
sleep(1)

In [5]:
write_lcdcmd(0xc0)
data = "Hello Python"
for c in data:
    write_data(ord(c))

sleep(1)

### 練習 在LCM模組上第一行奇數位址和第二行奇數位址顯示Hello
### hint: use 0x80 + col-1 and  0xc0 + col-1 to get the pos in first and second line 

In [53]:
write_lcdcmd(0x01)
col_1 = [1,3,5,7,9]
data = "Hello"
col_2 = [2,4,6,8,10]
for col, c in zip(col_1,data):
    write_lcdcmd(0x80 + col-1)
    write_data(ord(c))
for col, c in zip(col_2,data):
    write_lcdcmd(0xc0 + col-1)
    write_data(ord(c))

### 練習 做move_cursor(line, col), print_line_1(msg), print_line_2(msg), clear_line_1 & 2程式
### hint:  use this express:  (line-1) * 0x40 + 0x80 + (col-1) to get the position  

In [44]:
def move(line,col):
    pos = 0x80 + (line-1) * 0x40 + (col-1)
    write_lcdcmd(pos)

def print_line_1(msg):
    move(1,1)
    if type(msg) == str:
        for c in msg:
            write_data(ord(c))
            
def print_line_2(msg):
    move(2,1)
    if type(msg) == str:
        for c in msg:
            write_data(ord(c))

def clear_line_1():
    print_line_1(" " * 16)
    
def clear_line_2():
    print_line_2(" " * 16)





In [55]:
clear_line_1()
print_line_1("Hello Python")
clear_line_2()
print_line_2("Hello Python")

### 練習 配合subprocess.check_output() 呼叫指令ifconfig 得到IP 位置，並將其顯示在LCM模組之上。
### hint: use decode().split() to seperate each line and look for "addr:"

In [54]:
def get_ips():
    ip_data = check_output('ifconfig').decode().split()
    ip_list = []
    for item in ip_data:
        if "addr:" in item:
            ip_list.append(item[5:])
    return [i for i in ip_list if i]
get_ips()

['172.20.10.3', '127.0.0.1']

In [60]:
while(1):
        ip_list = get_ips()
        for i in ip_list :
            clear_line_1()
            print_line_1(i)
            sleep(1)

KeyboardInterrupt: 

In [104]:
def print_msg(msg):
    if type(msg) == str:
        for p in range(len(msg)):
            if p < 16:
                write_lcdcmd(0x80 + p)
                write_data(ord(msg[p]))
            elif 15 < p < 32:
                write_lcdcmd(0xc0 + p - 16)
                write_data(ord(msg[p]))
            else:
                print("error: it's out of bound")
                break

In [109]:
ip_list = get_ips()
clear_line_1()
clear_line_2()
print_msg(';'.join(ip_list))

### 練習 將以上所用function改寫成class, 並讓開機時候將 hostname 與 ip顯示在lcd上
### hint: $sudo nano /etc/rc.local and add python script before exit() 


'rpi'